In [116]:
import pandas as pd  
import statsmodels.api as sm  
import pylab as pl  
import numpy as np  
from patsy import dmatrix

In [117]:
csv = pd.read_csv("Pokemon.csv")
df = pd.DataFrame(csv)

In [118]:
# Add intercept

df['constante'] = 1.0

# Building a Type 1 how numeric type

index_types = []
i=0
for p in list(set(df['Type 1'])):
    i = i+1
    index_type = [p, i]
    index_types.append(index_type)


var_type1=[]
for typ in df['Type 1']:
    for ind_typ in index_types:
        if typ == ind_typ[0]:
            var_type1.append(ind_typ[1])
            
df['type1_num'] = var_type1

df.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,constante,type1_num
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1.0,17
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,1.0,17
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,1.0,17
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,1.0,17
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,1.0,12


In [119]:
# the variables to be used

mLogit = sm.Logit(df['Legendary'],df[['constante','Attack', 'Sp. Atk', "Sp. Def", 'Defense', "HP", "Speed",
                                      "Generation", "type1_num"]])

In [120]:
# fit the model and coefficients calculate

mLogit_res = mLogit.fit()

Optimization terminated successfully.
         Current function value: 0.099781
         Iterations 10


In [121]:
mLogit_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Legendary   No. Observations:                  800
Model:                          Logit   Df Residuals:                      791
Method:                           MLE   Df Model:                            8
Date:                Thu, 25 May 2017   Pseudo R-squ.:                  0.6459
Time:                        00:46:20   Log-Likelihood:                -79.825
converged:                       True   LL-Null:                       -225.45
                                        LLR p-value:                 2.997e-58
==============================================================================
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
constante    -26.7013      3.155     -8.463      0.000       -32.885   -20.517
Attack         0.0177      0.007      2.425      0.015         0.003     0.032
Sp. Atk        0.0393      0.008      4.912      0.000         0.024     0.055
Sp. Def        0.0462      0.010      4.834      0.000         0.027     0.065
Defense        0.0411      0.010      4.215      0.000         0.022     0.060
HP             0.0362      0.010      3.500      0.000         0.016     0.056
Speed          0.0572      0.010      5.503      0.000         0.037     0.078
Generation     0.4443      0.142      3.125      0.002         0.166     0.723
type1_num      0.0975      0.043      2.251      0.024         0.013     0.182
==============================================================================

Possibly complete quasi-separation: A fraction 0.38 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [122]:
print(mLogit_res.params) # vendo somente os coeficientes

constante    -26.701313
Attack         0.017726
Sp. Atk        0.039313
Sp. Def        0.046151
Defense        0.041083
HP             0.036210
Speed          0.057250
Generation     0.444273
type1_num      0.097546
dtype: float64


In [123]:
constant = mLogit_res.params[0]
att_cof = mLogit_res.params[1]
spatt_cof = mLogit_res.params[2]
spdef_cof = mLogit_res.params[3]
def_cof = mLogit_res.params[4]
hp_cof = mLogit_res.params[5]
sp_cof = mLogit_res.params[6]
gen_cof = mLogit_res.params[7]
type_cof = mLogit_res.params[8]

exp = -(constant+(att_cof*100)+(spatt_cof*120)+(spdef_cof*90)+(def_cof*90)
             +(hp_cof*150)+(sp_cof*80)+(gen_cof*2)+(type_cof*12))

In [124]:
# expoente aplicado a base da constante de euler

euler_exp = 2.718**exp

In [125]:
# Cálculo da probabilidade do evento ocorrer
probabilidade = (1/(1+euler_exp))


print("A pokemon that has \n Attack: 100 \n Sp. Att: 120 \n Sp. Def: 90 \n" 
      " Defense: 90 \n HP: 150 \n Speed: 80 \n Generation 2 \n Type: Fire")
print("Has the probability to be legendary of {}%".format(probabilidade*100))

A pokemon that has 
 Attack: 100 
 Sp. Att: 120 
 Sp. Def: 90 
 Defense: 90 
 HP: 150 
 Speed: 80 
 Generation 2 
 Type: Fire
Has the probability to be legendary of 42.81422350411192%


In [126]:
# calculate for database

atk_l = []
spa_l = []
spd_l = []
df_l = []
hp_l = []
sp_l = []
gen_l = []
typ_l = []

pred = []

for p in df[['Attack', 'Sp. Atk', "Sp. Def", 'Defense', "HP", "Speed",
                                      "Generation", "type1_num"]]:
    for value in df[p]:
        if p == 'Attack':
            atk_l.append(value)
        if p == 'Sp. Atk':
            spa_l.append(value)        
        if p == 'Defense':
            df_l.append(value)        
        if p == 'HP':
            hp_l.append(value)
        if p == 'Sp. Def':
            spd_l.append(value)
        if p == 'Speed':
            sp_l.append(value)
        if p == 'Generation':
            gen_l.append(value)
        if p == 'type1_num':
            typ_l.append(value)
            
# YEAH, I AM LAZZY, or idiot.

variables = np.column_stack((atk_l, spa_l, spd_l, df_l, hp_l, sp_l, gen_l, typ_l))            

for atk, spa, spd, dff, hp, sp, gen, typ in variables:            
            
    exp = -(constant+(att_cof*atk)+(spatt_cof*spa)+(spdef_cof*spd)+(def_cof*dff)
               +(hp_cof*hp)+(sp_cof*sp)+(gen_cof*gen)+(type_cof*typ))
    euler_exp = 2.718**exp
    probabilidade = (1/(1+euler_exp))

    pred.append(probabilidade*100)

In [127]:
probabilidade = pd.Series(pred)
df['probab'] = probabilidade 

In [128]:
# the last column have the probability of a pokemon to be legendary

df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,constante,type1_num,probab
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1.0,17,0.000643
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,1.0,17,0.021048
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,1.0,17,2.385263
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,1.0,17,50.955031
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,1.0,12,0.000338
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,1.0,12,0.015951
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False,1.0,12,2.569845
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False,1.0,12,34.550022
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,1.0,12,51.725116
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False,1.0,1,0.000117


In [129]:
# Verificar quantos foram os acertos

previstos = len(df[df['probab']>49])
reais = len(df[df['Legendary'] == True])
previstos_reais = len(df[df['Legendary'] == True][df['probab']>49])

print("The amount of pokemon predicted: {}".format(previstos))
print("The amount of pokemon legendary in database : {}".format(reais))

print("The amount of pokemons the model hit: {}".format(previstos_reais))

print("\n The model hit: {}%".format((previstos_reais*100)/reais))

The amount of pokemon predicted: 57
The amount of pokemon legendary in database : 65
The amount of pokemons the model hit: 43

 The model hit: 66.15384615384616%


/home/fandango/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
